In [2]:
import pandas as pd
from statsbombpy import sb
import streamlit as st
import warnings
warnings.filterwarnings('ignore')

In [3]:
competitions = sb.competitions()
womens_euro_competition = competitions[
    competitions["competition_name"] == "UEFA Women's Euro"
]
womens_euro_2022 = womens_euro_competition[
    womens_euro_competition["season_name"] == "2022"
]
euro_competition_id = womens_euro_2022.competition_id.unique()[0]
euro_season_id = womens_euro_2022.season_id.unique()[0]
match_ids = sb.matches(
    competition_id=euro_competition_id, season_id=euro_season_id
).match_id

In [7]:
matches=sb.matches(
    competition_id=euro_competition_id, season_id=euro_season_id
)
match_ids=matches[matches.match_date<conf.date_of_analysis].match_id

In [8]:
matches[matches.match_date<conf.date_of_analysis]

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3835331,2022-07-13,18:00:00.000,Europe - UEFA Women's Euro,2022,Sweden Women's,Switzerland Women's,2,1,available,...,2023-04-14T11:27:39.402579,2,Group Stage,Bramall Lane,Marta Huerta de Aza,Peter Gerhardsson,Nils Herbert Kromann Nielsen,1.1.0,2,2
1,3835324,2022-07-09,21:00:00.000,Europe - UEFA Women's Euro,2022,Netherlands Women's,Sweden Women's,1,1,available,...,2023-04-14T11:23:11.894822,1,Group Stage,Bramall Lane,Cheryl Foster,Mark Richard David Parsons,Peter Gerhardsson,1.1.0,2,2
2,3844384,2022-07-20,21:00:00.000,Europe - UEFA Women's Euro,2022,England Women's,Spain Women's,2,1,available,...,2023-04-26T02:03:52.207197,4,Quarter-finals,The American Express Community Stadium,Stéphanie Frappart,Sarina Glotzbach-Wiegman,Jorge Vilda,1.1.0,2,2
4,3845506,2022-07-26,21:00:00.000,Europe - UEFA Women's Euro,2022,England Women's,Sweden Women's,4,0,available,...,2023-04-26T00:44:10.275394,5,Semi-finals,Bramall Lane,Esther Staubli,Sarina Glotzbach-Wiegman,Peter Gerhardsson,1.1.0,2,2
5,3835335,2022-07-15,21:00:00.000,Europe - UEFA Women's Euro,2022,Northern Ireland,England Women's,0,5,available,...,2023-04-14T11:30:52.358455,3,Group Stage,St. Mary''s Stadium,Esther Staubli,Kenny Shiels,Sarina Glotzbach-Wiegman,1.1.0,2,2
6,3835323,2022-07-09,18:00:00.000,Europe - UEFA Women's Euro,2022,Portugal Women's,Switzerland Women's,2,2,available,...,2023-04-14T11:22:32.701542,1,Group Stage,Leigh Sports Village Stadium,Jana Adamkova,Francisco Miguel Conceição Roque Neto,Nils Herbert Kromann Nielsen,1.1.0,2,2
7,3835325,2022-07-10,21:00:00.000,Europe - UEFA Women's Euro,2022,France Women's,Italy Women's,5,1,available,...,2023-04-14T11:24:27.184039,1,Group Stage,AESSEAL New York Stadium,Rebecca Welch,Corinne Catherine Diacre,Milena Bertolini,1.1.0,2,2
8,3835320,2022-07-07,21:00:00.000,Europe - UEFA Women's Euro,2022,Norway Women's,Northern Ireland,4,1,available,...,2023-04-14T11:20:43.839249,1,Group Stage,St. Mary''s Stadium,Lina Lehtovaara,Martin Sjögren,Kenny Shiels,1.1.0,2,2
9,3845507,2022-07-27,21:00:00.000,Europe - UEFA Women's Euro,2022,Germany Women's,France Women's,2,1,available,...,2023-04-26T00:43:32.921353,5,Semi-finals,Stadium MK,Cheryl Foster,Martina Voss-Tecklenburg,Corinne Catherine Diacre,1.1.0,2,2
10,3844387,2022-07-23,21:00:00.000,Europe - UEFA Women's Euro,2022,France Women's,Netherlands Women's,1,0,available,...,2023-04-26T01:43:26.328990,4,Quarter-finals,AESSEAL New York Stadium,Ivana Martinčić,Corinne Catherine Diacre,Mark Richard David Parsons,1.1.0,2,2


In [2]:
def get_data(match_ids):
    event_data_tot=pd.DataFrame()
    for match_id in match_ids:
        event_data=sb.events(match_id=match_id)
        try:
            df_360=pd.read_json(f"/Users/borgwardt/Documents/repos/open-data/data/three-sixty/{match_id}.json")
        except:
            print(match_id)
        df_merged=pd.merge(event_data,df_360,how="left",left_on="id",right_on="event_uuid")
        event_data_tot=pd.concat([event_data_tot, df_merged], ignore_index=True)
    return event_data_tot
def preprocess_data(df_raw):
    df_preprocessed=df_raw.sort_values(["match_id","minute","timestamp"])
    df_preprocessed.reset_index(inplace=True)
    return df_preprocessed

def create_kpis(team_events):
    # Total goals
    goals = team_events[team_events['shot_outcome'] == 'Goal'].shape[0]
    
    # Total shots
    shots = len(team_events[team_events["type"] == "Shot"])
    # Total xg
    shot_statsbomb_xg=team_events["shot_statsbomb_xg"].sum()
    # Total passes
    passes = len(team_events[team_events["type"] == "Pass"])

    # Pass accuracy
    completed_passes = len(team_events[(team_events["type"] == "Pass") & (team_events["pass_outcome"].isnull())])
    pass_accuracy = (completed_passes / passes) * 100

    # Total duels won
    duels_won = len(team_events[(team_events["type"] == "Duel") & (team_events["duel_outcome"] == "Won")])

    # Total tackles
    tackles = len(team_events[team_events["type"] == "Tackle"])

    # Total interceptions
    interceptions = len(team_events[team_events["type"] == "Interception"])

    # Total clearances
    clearances = len(team_events[team_events["type"] == "Clearance"])

    # Percentage of possession 
    team_possession_seconds=team_events[(team_events['type']!="Pressure")].duration.sum()
    kpi_summary=pd.DataFrame({
        "goals": [goals],
        "shots": [shots],
        'shot_statsbomb_xg':[shot_statsbomb_xg],
        "passes": [passes],
        "pass_accuracy": [pass_accuracy],
        "duels_won": [duels_won],
        "tackles": [tackles],
        "interceptions": [interceptions],
        "clearances": [clearances],
        "possession_seconds": [team_possession_seconds]
    })
    return kpi_summary

In [3]:
competitions = sb.competitions()
womens_euro_competition = competitions[competitions['competition_name'] == "UEFA Women's Euro"]
womens_euro_2022 = womens_euro_competition[womens_euro_competition['season_name'] == '2022']
euro_competition_id=womens_euro_2022.competition_id.unique()[0]
euro_season_id=womens_euro_2022.season_id.unique()[0]
match_ids=sb.matches(competition_id=euro_competition_id, season_id=euro_season_id).match_id

In [4]:
df_raw=get_data(match_ids)

In [5]:
df_preprocessed=preprocess_data(df_raw)
df_preprocessed
df_kpis=df_preprocessed.groupby(["match_id","team"]).apply(create_kpis)
df_kpis.reset_index(level=2, drop=True,inplace=True)

In [6]:
df_kpis


goals  shots  shot_statsbomb_xg  passes  \
match_id team                                                        
3835319  Austria Women's       0      9           0.471990     392   
         England Women's       1     16           2.079074     568   
3835320  Northern Ireland      1      7           0.501850     308   
         Norway Women's        4     21           3.268670     566   
3835321  Spain Women's         4     35           3.075726     720   
...                          ...    ...                ...     ...   
3845506  Sweden Women's        0     11           1.161782     374   
3845507  France Women's        0     14           0.630761     461   
         Germany Women's       2     11           0.862559     441   
3847567  England Women's       2     13           1.680991     523   
         Germany Women's       1     15           1.758467     542   

                           pass_accuracy  duels_won  tackles  interceptions  \
match_id team                                                                 
3835319  Austria Women's       71.428571          5        0             16   
         England Women's       83.274648          2        0             24   
3835320  Northern Ireland      70.454545          3        0             18   
         Norway Women's        83.922261         10        0             16   
3835321  Spain Women's         87.638889          4        0             13   
...                                  ...        ...      ...            ...   
3845506  Sweden Women's        70.053476          4        0             17   
3845507  France Women's        74.837310          9        0             17   
         Germany Women's       77.777778          4        0             10   
3847567  England Women's       70.936902          6        0             20   
         Germany Women's       67.712177         10        0             13   

                           clearances  possession_seconds  
match_id team                                              
3835319  Austria Women's           17         1246.569215  
         England Women's           28         1735.339169  
3835320  Northern Ireland          10         1075.754234  
         Norway Women's            13         1671.604761  
3835321  Spain Women's             18         2124.495201  
...                               ...                 ...  
3845506  Sweden Women's            11         1039.787967  
3845507  France Women's            13         1427.588485  
         Germany Women's           23         1437.772512  
3847567  England Women's           36         1571.562727  
         Germany Women's           26         1565.956965  

[62 rows x 10 columns]

In [7]:
df_preprocessed#.index

,level_0,50_50,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,block_offensive,block_save_block,carry_end_location,clearance_aerial_won,clearance_body_part,...,goalkeeper_punched_out,goalkeeper_shot_saved_to_post,pass_miscommunication,shot_saved_to_post,shot_open_goal,foul_committed_penalty,foul_won_penalty,goalkeeper_success_in_play,half_start_late_video_start,shot_redirect
0,101726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,101727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105152,13303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105153,13365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,Head,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105154,12396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105155,14996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
pd.DataFrame({
        "shots": 1,
        'shot_statsbomb_xg':1,
        "passes": 1,
        "pass_accuracy": 1,
        "duels_won": 1,
        "tackles": 1,
        "interceptions": 1,
        "clearances": 1,
        "possession_seconds": 1
    })

ValueError: If using all scalar values, you must pass an index

In [9]:
pd.read_json("/Users/borgwardt/Documents/repos/open-data/data/three-sixty/3835342.json")

,event_uuid,visible_area,freeze_frame
0,fde0e241-0ff3-4faf-9806-bbcd9d0bfb28,"[39.6248103518052, 77.602435306958, 0.0, 6.433...","[{'teammate': True, 'actor': False, 'keeper': ..."
1,fa701824-c7c5-4597-8b11-68ec53e0f68e,"[37.0973922997033, 75.3607031690491, 0.0, 18.6...","[{'teammate': True, 'actor': False, 'keeper': ..."
2,6f1761f2-f827-4c57-a960-b64b14b407f9,"[37.0973922997033, 75.3607031690491, 0.0, 18.6...","[{'teammate': True, 'actor': False, 'keeper': ..."
3,59c03233-bf02-407e-8c62-39ab4c9d8795,"[13.699331198296, 80.0, 45.5966437365325, 8.34...","[{'teammate': True, 'actor': False, 'keeper': ..."
4,de12a14f-f7c9-407a-9250-49ec6689fba8,"[32.8918846791763, 76.423516510925, 0.0, 28.94...","[{'teammate': True, 'actor': False, 'keeper': ..."
...,...,...,...
2797,be94d3f3-80af-460b-a4ac-90d8bddb0882,"[107.715623164502, 80.0, 82.4999149834406, 27....","[{'teammate': False, 'actor': False, 'keeper':..."
2798,58f3c71a-60bf-4354-8e58-b7fd73590604,"[15.9252544426531, 60.9847502771868, 0.0, 45.9...","[{'teammate': True, 'actor': False, 'keeper': ..."
2799,7aeb57d4-6433-4772-9e90-fc9138e58f40,"[15.9252544426531, 60.9847502771868, 0.0, 45.9...","[{'teammate': False, 'actor': False, 'keeper':..."
2800,7399d8d7-a0a1-4a2d-b256-c17acab1efc0,"[113.821253405624, 80.0, 84.7732651830841, 25....","[{'teammate': True, 'actor': False, 'keeper': ..."
